# idact - Prometheus sandbox

## Initial setup

Add `idact` to path:

In [1]:
import sys
import os
import bitmath
import getpass
import contextlib
import fabric
import logging
from pprint import pprint

def append_idact_path():
    idact_path = os.path.realpath(os.path.join(os.getcwd(), '../'))
    sys.path.append(idact_path)
append_idact_path()

from idact import *
from idact.detail.auth.set_password import set_password

os.environ['IDACT_KEY_LOCATION'] = os.path.join(os.getcwd(), '../.notebook-ssh')
# os.environ['IDACT_KEY_LOCATION'] = os.path.expanduser('~/.ssh')
if not os.path.exists(os.environ['IDACT_KEY_LOCATION']):
    os.mkdir(os.environ['IDACT_KEY_LOCATION'])

USER = 'plggarstka'

Hide debug information, setup context manager stack (for testing purposes)

## Add cluster (only first run)

In [2]:
key = KeyType.RSA  # Generate RSA key
# key = os.path.join(os.path.expanduser('~/.ssh'), 'id_rsa')

In [3]:
cluster = add_cluster(name="pro",
                      user=USER,
                      host="pro.cyfronet.pl",
                      port=22,
                      auth=AuthMethod.PUBLIC_KEY,
                      key=key,
                      install_key=True,
                      scratch="$SCRATCH")
save_environment('.idact-env')

2018-10-06 20:01:05 INFO: Generating public-private key pair.


## Load cluster (subsequent runs)

In [4]:
load_environment('.idact-env')
cluster = show_cluster("pro")
cluster

Cluster(pro.cyfronet.pl, 22, plggarstka, auth=AuthMethod.PUBLIC_KEY, key='E:\\shared\\uni\\eng-project\\notebooks\\../.notebook-ssh\\id_rsa_se', install_key=True, disable_sshd=False)

In [5]:
set_log_level(logging.INFO)
#set_log_level(logging.DEBUG)
save_environment('.idact-env')

In [6]:
node = cluster.get_access_node()
node

Node(pro.cyfronet.pl:22, None)

On your first action, you will be asked for a password to install the key.
You can connect explicitly (optional) to do this right now:

In [7]:
node.connect()

2018-10-06 20:01:08 INFO: Installing key using password authentication.
Password for plggarstka@pro.cyfronet.pl:22: 


In [8]:
node.run('whoami')

'plggarstka'

In [9]:
node.run('hostname')

'login01.pro.cyfronet.pl'

## Allocate nodes

In [10]:
nodes = cluster.allocate_nodes(nodes=2,
                               cores=2,
                               memory_per_node=bitmath.GiB(10),
                               walltime=Walltime(minutes=20),
                               native_args={
                                   '--partition': 'plgrid-testing',
                                   '--account': 'intdata'
                               })

2018-10-06 20:01:22 INFO: Creating the ssh directory.


In [11]:
nodes

Nodes([Node(NotAllocated),Node(NotAllocated)])

In [12]:
nodes.wait()
nodes

Nodes([Node(p0721:49301, 2018-10-06 18:21:26.583210+00:00),Node(p0727:40770, 2018-10-06 18:21:26.583210+00:00)])

## Run commands

In [13]:
nodes[0].run('whoami')

'plggarstka'

In [14]:
nodes[0].run('hostname')

'p0721'

In [15]:
nodes[1].run('squeue')

'JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)\n          13581290 plgrid-te     wrap plggarst  R       0:04      2 p[0721,0727]'

In [16]:
nodes[1].run('hostname')

'p0727'

## Examine node resources

In [17]:
nodes[0].resources.memory_total

GiB(10.0)

In [18]:
nodes[0].resources.memory_usage

GiB(0.02249908447265625)

In [19]:
nodes[0].resources.cpu_cores

2

In [20]:
nodes[0].resources.cpu_usage

0.0

## Tunnel

In [21]:
tunnel = nodes[0].tunnel(here=9000, there=10000)

In [22]:
tunnel

MultiHopTunnel(9000:10000)

In [23]:
tunnel.close()

## Deploy notebook

One-time config step (cluster-specific):

In [24]:
cluster.config.setup_actions.jupyter = ['module load plgrid/tools/python-intel/3.6.2']
save_environment('.idact-env')

To run Jupyter Notebook on the cluster:

In [25]:
nb = nodes[0].deploy_notebook(local_port=8080)
nb

2018-10-06 20:01:42 WARNING: Exception: Cannot run task., retry 1/5.


JupyterDeployment(8080 -> Node(p0721:49301, 2018-10-06 18:21:26.583210+00:00))

In [26]:
nodes[0].resources.memory_usage

GiB(0.081085205078125)

In [27]:
nb.local_port

8080

To open the deployed notebook server in a new tab:

In [28]:
nb.open_in_browser()

In [29]:
nodes[0].resources.memory_usage

GiB(0.08215713500976562)

In [30]:
nb.cancel()

2018-10-06 20:02:30 INFO: Cancelling Jupyter deployment.


## Deploy Dask

One-time config step (cluster-specific):

In [31]:
cluster.config.setup_actions.dask = ['module load plgrid/tools/python-intel/3.6.2']
cluster.config.scratch = '$SCRATCH'
save_environment('.idact-env')

In [32]:
dd = deploy_dask(nodes)
dd

2018-10-06 20:02:36 INFO: Deploying Dask on 2 nodes.
2018-10-06 20:02:36 INFO: Connecting to p0721:49301 (1/2).
2018-10-06 20:02:36 INFO: Connecting to p0727:40770 (2/2).
2018-10-06 20:02:37 INFO: Deploying scheduler on the first node: p0721.
2018-10-06 20:02:46 WARNING: Exception: Unable to determine scheduler address., retry 1/5.
2018-10-06 20:02:53 INFO: Checking scheduler connectivity from p0721 (1/2).
2018-10-06 20:02:53 INFO: Checking scheduler connectivity from p0727 (2/2).
2018-10-06 20:02:53 INFO: Deploying workers.
2018-10-06 20:02:53 INFO: Deploying worker 1/2.
2018-10-06 20:03:03 WARNING: Exception: Cannot run task., retry 1/5.
2018-10-06 20:03:10 INFO: Deploying worker 2/2.
2018-10-06 20:03:20 INFO: Validating worker 1/2.
2018-10-06 20:03:20 INFO: Validating worker 2/2.


DaskDeployment(scheduler=tcp://localhost:52179/tcp://172.20.66.211:39995, workers=2)

In [33]:
nodes[0].resources.memory_usage

GiB(0.36870574951171875)

Get Dask client:

In [34]:
client = dd.get_client()
client

Client Scheduler: tcp://localhost:52179 Dashboard: http://localhost:37308/status,Cluster Workers: 2 Cores: 4 Memory: 21.47 GB


In [35]:
nodes[0].resources.cpu_usage

8.0

Computation will work only if Python and library versions match:

In [36]:
#x = client.submit(lambda: value + 1, 10)
#x.result() == 11

Diagnostics servers are tunnelled:

In [37]:
dd.diagnostics.addresses

['http://localhost:37308', 'http://localhost:52191', 'http://localhost:52197']

To open diagnostics servers in new tabs:

In [38]:
dd.diagnostics.open_all()

In [39]:
dd.cancel()

2018-10-06 20:03:54 INFO: Cancelling worker deployment on p0727.
2018-10-06 20:04:01 INFO: Cancelling worker deployment on p0721.
2018-10-06 20:04:08 INFO: Cancelling scheduler deployment on p0721.


## Close

In [40]:
nodes.running()

True

In [41]:
nodes.cancel()

2018-10-06 20:04:17 INFO: Cancelling job 13581290.


In [42]:
nodes.running()

False

In [43]:
node.run('squeue')

'JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)\n          13581290 plgrid-te     wrap plggarst CG       2:53      2 p[0721,0727]'

## Push and pull the environment

When working on a cluster, it may be useful to synchronize idact config with the local machine. Pushing the environment will merge the local environment into the remote environment.

In [44]:
push_environment(cluster, path='~/.idact-notebook-env')

2018-10-06 20:04:22 INFO: Pushing the environment to cluster.
2018-10-06 20:04:22 ERROR: Failure: Getting file from node pro.cyfronet.pl: /net/people/plggarstka/.idact-notebook-env
2018-10-06 20:04:22 ERROR: Failure: Deserializing the environment from cluster.
2018-10-06 20:04:22 INFO: Remote environment is missing, current environment will be copied to cluster.


In [45]:
print(node.run('cat ~/.idact-notebook-env'))

{
    "clusters": {
        "pro": {
            "auth": "PUBLIC_KEY",
            "disableSshd": false,
            "host": "pro.cyfronet.pl",
            "installKey": true,
            "key": null,
            "port": 22,
            "portInfoRetries": 5,
            "scratch": "$SCRATCH",
            "setupActions": {
                "dask": [
                    "module load plgrid/tools/python-intel/3.6.2"
                ],
                "jupyter": [
                    "module load plgrid/tools/python-intel/3.6.2"
                ]
            },
            "user": "plggarstka"
        }
    },
    "logLevel": 20
}


The reverse operation is pulling the environment, which merges the remote environment into the local environment. Machine-specific information like the private key path is skipped when pushing or pulling.

In [46]:
pull_environment(cluster, path='~/.idact-notebook-env')

2018-10-06 20:04:23 INFO: Pulling the environment from cluster.


The 'path' parameter is optional. It defaults to ~/.idact.conf, or the value of the remote IDACT_CONFIG_PATH environment variable.

In [47]:
node.run('rm -v ~/.idact-notebook-env')

'removed ‘/net/people/plggarstka/.idact-notebook-env’'

## Remove cluster

A cluster can be removed from the environment.

In [48]:
add_cluster(name='fake',
            user='fakeuser',
            host='fakehost',
            port=2222)

2018-10-06 20:04:24 INFO: No auth method specified, defaulting to password-based.


Cluster(fakehost, 2222, fakeuser, auth=AuthMethod.ASK, key=None, install_key=True, disable_sshd=False)

In [49]:
show_clusters()

{'pro': Cluster(pro.cyfronet.pl, 22, plggarstka, auth=AuthMethod.PUBLIC_KEY, key='E:\\shared\\uni\\eng-project\\notebooks\\../.notebook-ssh\\id_rsa_se', install_key=False, disable_sshd=False),
 'fake': Cluster(fakehost, 2222, fakeuser, auth=AuthMethod.ASK, key=None, install_key=True, disable_sshd=False)}

In [50]:
remove_cluster('fake')

In [51]:
show_clusters()

{'pro': Cluster(pro.cyfronet.pl, 22, plggarstka, auth=AuthMethod.PUBLIC_KEY, key='E:\\shared\\uni\\eng-project\\notebooks\\../.notebook-ssh\\id_rsa_se', install_key=False, disable_sshd=False)}